# FastMCP 클라이언트 튜토리얼 

## 개요

이 튜토리얼은 FastMCP(Model Context Protocol) 클라이언트를 사용하여 MCP 서버와 통신하는 방법을 단계별로 설명합니다. MCP는 AI 모델이 외부 도구와 리소스에 접근할 수 있게 해주는 표준 프로토콜입니다.

우리가 구현한 서버는 다음 기능을 제공합니다:
- **리소스**: 현재 디렉토리의 파일 목록 조회
- **도구**: 간단한 계산기와 스크린샷 촬영 기능

## 2. 기본 라이브러리 Import
### Import 설명
- `asyncio`: 비동기 통신을 위한 Python 표준 라이브러리
- `MultiServerMCPClient`: 여러 MCP 서버와 동시 연결 가능한 클라이언트
- `json`: 서버 응답 데이터 파싱용
- `base64`: 이미지 데이터 디코딩용
- `cv2`, `PIL`, `numpy`: 이미지 처리 및 표시용

In [1]:
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
import json
import base64
import cv2
import numpy as np
from PIL import Image
import io

## 3. MCP 서버 연결 설정
### 연결 방식 설명
- **SSE (Server-Sent Events)**: 실시간 단방향 통신 방식
- **비동기 연결**: `await`를 사용하여 논블로킹 방식으로 처리
- **안정화 대기**: 네트워크 지연을 고려한 2초 대기

In [ ]:
async def connect_to_server():
    """MCP 서버에 연결"""
    server_name = "desktop_server"
    
    client = MultiServerMCPClient({
        server_name: {
            "url": "http://localhost:8006/sse",
            "transport": "sse",
        }
    })
    
    await client.__aenter__()
    print("MCP 서버에 연결됨")
    
    # 연결 안정화 대기
    await asyncio.sleep(2)
    
    return client, server_name

# 실행
client, server_name = await connect_to_server()

MCP 서버에 연결됨


Error in sse_reader: 


## 4. 현재 디렉토리 리소스 확인
### 리소스 개념 설명
- **리소스**: 서버가 제공하는 읽기 전용 데이터 (파일, 설정 등)
- **URI 스키마**: `dir://current`는 현재 디렉토리를 의미하는 커스텀 URI
- **JSON 응답**: 구조화된 데이터로 파일 경로와 목록 정보 포함

In [3]:
async def check_directory():
    """현재 디렉토리의 파일 목록 조회"""
    print("=== 현재 디렉토리 리소스 ===")
    
    current_resources = await client.get_resources(server_name, ["dir://current"])
    if current_resources:
        data = json.loads(current_resources[0].data)
        print(f"경로: {data['path']}")
        print(f"파일 개수: {data['count']}개")
        print("파일 목록:")
        for i, file in enumerate(data['files'][:10], 1):  # 처음 10개만 표시
            print(f"  {i}. {file}")
        if data['count'] > 10:
            print(f"  ... 및 {data['count'] - 10}개 더")
    else:
        print("현재 디렉토리 리소스를 찾을 수 없습니다")

# 실행
await check_directory()

=== 현재 디렉토리 리소스 ===
경로: C:\workspace\KAMP_로컬LLM
파일 개수: 32개
파일 목록:
  1. C:\workspace\KAMP_로컬LLM\.ipynb_checkpoints
  2. C:\workspace\KAMP_로컬LLM\Agent_example_수강용.ipynb
  3. C:\workspace\KAMP_로컬LLM\datasets
  4. C:\workspace\KAMP_로컬LLM\deepseek-coder-v2_example_수강용.ipynb
  5. C:\workspace\KAMP_로컬LLM\Deepseek-R1-Distill-Llama-8b_DoRA_alpaca_50
  6. C:\workspace\KAMP_로컬LLM\Deepseek-R1-Distill-Llama-8b_LoRA_alpace_50
  7. C:\workspace\KAMP_로컬LLM\document
  8. C:\workspace\KAMP_로컬LLM\document_실습용
  9. C:\workspace\KAMP_로컬LLM\faiss_index
  10. C:\workspace\KAMP_로컬LLM\images
  ... 및 22개 더


## 5. 사용 가능한 도구 확인
### 도구(Tool) vs 리소스(Resource)
- **도구(Tool)**: 실행 가능한 함수 (계산, 스크린샷 등)
- **리소스(Resource)**: 정적 데이터 (파일 목록, 설정 등)
- **동기 vs 비동기**: 도구 목록 조회는 동기, 실행은 비동기

In [4]:
def check_available_tools():
    """사용 가능한 도구 목록 확인"""
    print("=== 사용 가능한 도구 ===")
    
    tools = client.get_tools()
    print(f"총 {len(tools)}개의 도구가 있습니다:")
    
    for i, tool in enumerate(tools, 1):
        print(f"  {i}. {tool.name}: {tool.description}")
    
    return tools

# 실행
tools = check_available_tools()

=== 사용 가능한 도구 ===
총 2개의 도구가 있습니다:
  1. add: Add two numbers
  2. take_screenshot: Take a screenshot of the user's screen


## 6. 덧셈 계산기 도구 테스트
### 계산기 도구 상세 분석
- **비동기 실행**: `await tool.coroutine()`로 도구 실행
- **매개변수 전달**: 키워드 인자로 `a`, `b` 값 전달
- **응답 형태**: 서버가 JSON 문자열을 튜플로 감싸서 반환

In [5]:
async def test_calculator(a=10, b=5):
    """덧셈 계산기 테스트"""
    print(f"=== 계산기 테스트: {a} + {b} ===")
    
    add_tool = next((t for t in tools if t.name == "add"), None)
    if add_tool:
        add_result = await add_tool.coroutine(a=a, b=b)
        
        # 튜플 처리
        if isinstance(add_result, tuple):
            add_data = json.loads(add_result[0])
        else:
            add_data = json.loads(add_result)
        
        print(f"결과: {add_data['operation']} = {add_data['result']}")
    else:
        print("add 도구를 찾을 수 없습니다")

# 실행
await test_calculator(10, 5)
await test_calculator(100, 200)

=== 계산기 테스트: 10 + 5 ===
결과: 10 + 5 = 15
=== 계산기 테스트: 100 + 200 ===
결과: 100 + 200 = 300


## 7. 스크린샷 촬영 및 표시
### 스크린샷 처리 과정 상세 설명

1. **서버 처리**: pyautogui로 화면 캡처 → JPEG 압축 → base64 인코딩
2. **클라이언트 처리**: base64 디코딩 → PIL 이미지 → numpy 배열 → OpenCV 표시
3. **색상 변환**: RGB (PIL) → BGR (OpenCV) 변환 필요
4. **크기 조정**: 원본 비율 유지하면서 600x400 안에 맞춤
5. **메모리 관리**: 이미지 처리 후 OpenCV 창 정리

In [6]:
async def take_and_show_screenshot():
    """스크린샷 촬영 및 표시"""
    print("=== 스크린샷 테스트 ===")
    
    screenshot_tool = next((t for t in tools if t.name == "take_screenshot"), None)
    if not screenshot_tool:
        print("take_screenshot 도구를 찾을 수 없습니다")
        return
    
    print("스크린샷 촬영 중...")
    screenshot_result = await screenshot_tool.coroutine()
    
    # 튜플 처리
    if isinstance(screenshot_result, tuple):
        screenshot_data = json.loads(screenshot_result[0])
    else:
        screenshot_data = json.loads(screenshot_result)
    
    print(f"스크린샷 완료: {screenshot_data['size_bytes']} bytes")
    
    try:
        # base64 디코딩
        image_data = base64.b64decode(screenshot_data['image_base64'])
        image = Image.open(io.BytesIO(image_data))
        
        # PIL 이미지를 numpy 배열로 변환
        img_array = np.array(image)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        # 원본 크기
        original_height, original_width = img_array.shape[:2]
        print(f"원본 크기: {original_width}x{original_height}")
        
        # 600x400 안에 맞추면서 비율 유지
        target_width, target_height = 600, 400
        ratio = min(target_width / original_width, target_height / original_height)
        new_width = int(original_width * ratio)
        new_height = int(original_height * ratio)
        
        # 비율을 유지하며 리사이즈
        img_resized = cv2.resize(img_array, (new_width, new_height))
        
        # OpenCV로 표시
        cv2.imshow('Screenshot', img_resized)
        print(f"스크린샷이 표시되었습니다 ({new_width}x{new_height})")
        print("아무 키나 누르면 창이 닫힙니다")
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    except Exception as e:
        print(f"이미지 표시 오류: {e}")

# 실행
await take_and_show_screenshot()

=== 스크린샷 테스트 ===
스크린샷 촬영 중...
스크린샷 완료: 190519 bytes
원본 크기: 1920x1080
스크린샷이 표시되었습니다 (600x337)
아무 키나 누르면 창이 닫힙니다
